In [2]:
!pip install -q transformers sentencepiece

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

In [3]:
model_nllb = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M").to("cuda")
tokenizer_nllb = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

In [4]:
def translate_nllb(text, source_lang: str, target_lang: str) -> str:
    tokenizer_nllb.src_lang = source_lang
    inputs = tokenizer_nllb(
        text,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=128
    ).to("cuda")

    bos_token_id = tokenizer_nllb.convert_tokens_to_ids(target_lang)

    with torch.no_grad():
        output = model_nllb.generate(
            **inputs,
            forced_bos_token_id=bos_token_id,
            max_new_tokens=128,
            num_beams=4,
            repetition_penalty=1.2,
            early_stopping=True
        )

    return tokenizer_nllb.decode(output[0], skip_special_tokens=True)



In [5]:
def bt_enko(text):
    try:
        en = translate_nllb(text, "kor_Hang", "eng_Latn")
        ko = translate_nllb(en, "eng_Latn", "kor_Hang")
        return ko
    except Exception as e:
        return f"[ERROR]: {str(e)}"

def bt_deko(text):
    try:
        de = translate_nllb(text, "kor_Hang", "deu_Latn")
        ko = translate_nllb(de, "deu_Latn", "kor_Hang")
        return ko
    except Exception as e:
        return f"[ERROR]: {str(e)}"

def bt_frko(text):
    try:
        fr = translate_nllb(text, "kor_Hang", "fra_Latn")
        ko = translate_nllb(fr, "fra_Latn", "kor_Hang")
        return ko
    except Exception as e:
        return f"[ERROR]: {str(e)}"

def bt_jako(text):
    try:
        ja = translate_nllb(text, "kor_Hang", "jpn_Jpan")
        ko = translate_nllb(ja, "jpn_Jpan", "kor_Hang")
        return ko
    except Exception as e:
        return f"[ERROR]: {str(e)}"


In [27]:
import pandas as pd
from google.colab import files

# 사용자에게 파일 업로드 요청
uploaded = files.upload()

# 업로드된 첫 번째 파일 이름 추출
filename = next(iter(uploaded))

# 데이터 로딩
df = pd.read_csv(filename)[["Q", "A"]].dropna().reset_index(drop=True)

# 샘플 추출
sample_df = df.sample(5, random_state=42).reset_index(drop=True)

Saving train (1).csv to train (1).csv


In [28]:
!pip install -q sentence-transformers

from sentence_transformers import SentenceTransformer, util

model_sbert = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")  # 한국어 포함


In [37]:
def is_similar(original: str, backtranslated: str, threshold=0.635):
    emb1 = model_sbert.encode(original, convert_to_tensor=True)
    emb2 = model_sbert.encode(backtranslated, convert_to_tensor=True)
    score = util.pytorch_cos_sim(emb1, emb2).item()
    return score >= threshold

In [20]:
aug = bt_jako("눈은 기본이죠.")
print(aug, is_similar("눈은 기본이죠.", aug))

시각은 기본적인 것입니다. 0.8224431276321411


In [23]:
# 샘플 5개 추출
sample_df = df.sample(10, random_state=70).reset_index(drop=True)

# 실험
for i, row in sample_df.iterrows():
    print(f"\n🟩 샘플 {i+1}")

    for field in ["Q", "A"]:
        original = row[field]
        print(f"{field}: {original}")

        for tag, bt_func in [("en", bt_enko), ("de", bt_deko), ("fr", bt_frko), ("ja", bt_jako)]:
            aug = bt_func(original)
            sim = is_similar(original, aug)  # float 값을 리턴하도록 구현되어 있다고 가정
            print(f"  ↳ {tag}→ko: {aug}  (유사도: {sim:.4f})")




🟩 샘플 1
Q: 성덕 되면 내 마음도 편해 지겠지
  ↳ en→ko: 내가 성인이면 마음이 안심할 거야  (유사도: 0.9231)
  ↳ de→ko: 솔직히 말해서, 저는 편안하게 느껴집니다.  (유사도: 0.4753)
  ↳ fr→ko: 내가 성인이 될 때 더 편안하게 느껴질 거야  (유사도: 0.8650)
  ↳ ja→ko: 신자가 되면 마음이 안심해질 거야  (유사도: 0.9087)
A: 당신이 잘돼야 성덕이 될수있겠죠.
  ↳ en→ko: 좋은 사람이 되어야, 성인이 될 수 있다.  (유사도: 0.5158)
  ↳ de→ko: 넌 좋은 사람이야, 솔직할 수 있어.  (유사도: 0.6614)
  ↳ fr→ko: 당신은 좋은 사람이어야 합니다. 당신은 거룩한 사람이 될 수 있습니다.  (유사도: 0.6887)
  ↳ ja→ko: 당신은 좋은 사람이 되고, 성인이 될 수 있습니다.  (유사도: 0.6390)

🟩 샘플 2
Q: 나는 나약한 존재
  ↳ en→ko: 난 약한 존재야  (유사도: 0.7663)
  ↳ de→ko: 난 약한 존재야  (유사도: 0.7663)
  ↳ fr→ko: 난 약한 존재야.  (유사도: 0.9700)
  ↳ ja→ko: 난 약한 존재야  (유사도: 0.7663)
A: 절대 그렇지 않아요.
  ↳ en→ko: 그건 사실이 아닙니다.  (유사도: 0.7228)
  ↳ de→ko: 그건 사실이 아닙니다.  (유사도: 0.7228)
  ↳ fr→ko: 그런 일은 결코 일어나지 않습니다.  (유사도: 0.7253)
  ↳ ja→ko: 이런 일은 결코 일어나지 않습니다.  (유사도: 0.7196)

🟩 샘플 3
Q: 낚시 좋아하는 남자 어때?
  ↳ en→ko: 낚시를 좋아하는 놈은 어때?  (유사도: 0.9625)
  ↳ de→ko: 낚시를 좋아하는 남자가 어떨까요?  (유사도: 0.9648)
  ↳ fr→ko: 낚시를 좋아하는 사람이면 어떨까요?  (유사도: 0.8749)
  ↳ ja→ko: 낚시를 좋아하는 남자?  (유사도: 0.

In [40]:
from tqdm import tqdm

# 결과 저장 리스트
augmented_q = []
augmented_a = []

print("데이터 크기:", df.shape)
tqdm.pandas()

for _, row in tqdm(df.iterrows(), total=len(df)):
    q = row["Q"]
    a = row["A"]

    try:
        q_aug = bt_enko(q)
        if is_similar(q, q_aug):
            augmented_q.append(q_aug)
            augmented_a.append(a)

        a_aug = bt_enko(a)
        if is_similar(a, a_aug):
            augmented_q.append(q)
            augmented_a.append(a_aug)

    except Exception:
        continue


데이터 크기: (6957, 2)


100%|██████████| 6957/6957 [1:20:45<00:00,  1.44it/s]


In [41]:
# 새로운 증강 결과 DataFrame 생성
aug_df = pd.DataFrame({
    "Q": augmented_q,
    "A": augmented_a
})

print("증강된 샘플 개수:", aug_df.shape)
aug_df.head(20)


증강된 샘플 개수: (9263, 2)


,Q,A
0,좋아하는 사람 생기면 자꾸 따라하게 돼.,이것은 자연스러운 현상입니다.
1,내 남자친구는 나를 모욕하고 있어,순간 실수할 수 있겠다 판단되면 용서하고 기회를 주세요.
2,남자친구가 욕함,순식간에 실수를 할 수 있어 판단되면 용서하고 기회를 줘
3,새로운 운동화 필요해요,좋은 곳으로 갈 수 있도록 인도해줄 거예요.
4,한 달 동안 문자 보냈어요,이제 기억 속에서 지우는게 좋겠어요.
5,문자 보내놓은지 어언 한 달.,이제 제 기억에서 지우고 싶습니다.
6,"아미티, 가야 돼?",필수는 아니에요.
7,엠티 꼭 가야해?,그럴 필요는 없어요.
8,썸 타는 사이가 이렇게 편해도 되나?,너무 편안하다고 말해
9,썸 타는 사람한테 영통 걸까?,걸을 수 있는지 물어보세요.


In [42]:
# 저장
output_path = "ChatbotData_backtranslated.csv"
aug_df.to_csv(output_path, index=False, encoding="utf-8-sig")

# Google Drive 저장
from google.colab import drive, files
drive.mount('/content/drive')
!cp {output_path} /content/drive/MyDrive/

# 다운로드 링크 생성
files.download(output_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>